#Pandas Express 
###An express guide to becoming a Kung Fu Pandas master

<img src="http://vignette1.wikia.nocookie.net/kungfupanda/images/8/88/Po2.jpg/revision/latest?cb=20100726062228" width="300"/>

After defeating the evil snow leopard Tai Lung, our favorite kung fu panda master Po returns to the Valley of Peace to help his father Mr. Ping with his noodle restaurant. Mr. Ping's noodle restaurant hasn't been doing so well, so Po is determined to help his dad figure out what he can do to improve his restaurant. Luckily, Po has been trained in the revered and ancient Python style of Shaolin martial arts and will analyze a dataset from Yelp to save his father's restaurant, like a true Kung Fu Pandas master.

###The Tools
This tutorial will walk you through doing some basic data cleaning and exploratory analysis with Pandas and a suite of other Python data analysis tools. Below are a few of the tools we will be using:

* [numpy](http://docs.scipy.org/doc/numpy-dev/user/index.html), for arrays
* [pandas](http://pandas.pydata.org/), for data frames
* [matplotlib](http://matplotlib.org/), for plotting
* [seaborn](http://stanford.edu/~mwaskom/software/seaborn/), for making plots pretty
* [statsmodels](http://statsmodels.sourceforge.net/), for statistical analysis
* [sklearn](http://scikit-learn.org), for machine learning

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels
import sklearn 

# iPython command to format matplotlib plots
%matplotlib inline 

If you have trouble importing any of the packages, you might need to install it first from the website or, if you're on Mac OS or ubuntu, from the console with: `pip install <name of package>`

###The Dataset
We will be using a dataset of Yelp reviews provided by the [Yelp Dataset Challenge](http://www.yelp.com/dataset_challenge). The download consists of the following files in JSON format:
* business.json - information on businesses
* review.json - text and metadata of reviews
* tip.json - text and metadata of tips
* user.json - information on users
* checkin.json - number of checkins at each business

In this tutorial, we will be primarily focused on the business.json file

###Loading in and cleaning the data
The Yelp dataset is in JSON format, [which you can read about here](https://en.wikipedia.org/wiki/JSON) if you are at all interested. Luckily, Python (and most other programming languages) has packages for parsing and reading JSON file formats. In this case, we are going to use Python's JSON file reader to read in the file and then convert it to a Pandas DataFrame. 

(Note: Pandas also has a [`read_json`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_json.html) function that reads in a JSON file into a DataFrame directly, but in this case, we had to do a little cleanup to remove trailing whitespace at the end of each line, so we didn't use that particular function)

####Load in data

In [ ]:
import json

'''
load_data(filepath) 
Given a filepath to a JSON file, loads in the file and formats the JSON
'''
def load_data(filepath):
    data = []
    
    # Open file and read in line by line
    with open(filepath) as file:
        for line in file:
            # Strip out trailing whitespace at the end of the line
            data.append(json.loads(line.rstrip()))

    return data

In [ ]:
data = load_data('data/business.json')

In [ ]:
business_df = pd.DataFrame.from_dict(data)

####Now let's take a peek inside
The [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/api.html#dataframe) has a full list of functions, but below are some helpful ones for doing some initial poking around. 

In [ ]:
'''
head(n=5)
Returns first n rows
'''
business_df.head(5)

In [ ]:
'''
info(verbose=None, buf=None, max_cols=None, memory_usage=None, null_counts=None)
Concise summary of a DataFrame.
'''
business_df.info(verbose=True, null_counts=True)

In [ ]:
'''
describe(percentile_width=None, percentiles=None, include=None, exclude=None)
Generate various summary statistics, excluding NaN values.
'''
business_df.describe()

In [ ]:
'''
shape
Attribute of a DataFrame as (rows, columns)
'''
business_df.shape

Some of the most commonly used operations on Dataframes are various types of indexing, filtering, and slicing. Pandas has a number of different ways to do these operations ([check out this whole page of documentation about it](http://pandas.pydata.org/pandas-docs/stable/indexing.html)), but below are a few examples.

In [ ]:
# Select a column (returns a Series)
business_df['name'].head()

In [ ]:
# Select multiple columns (returns a DataFrame)
business_df[['business_id', 'name']].head()

In [ ]:
# Filter a column on a value
business_df[business_df['name'] == 'Panda Express'].head()

###Cleaning and formatting columns

To scope out the competition, Po might be interesting in filtering the dataset to see what other Chinese restuarants are out there and see what types of attributes they offer (e.g. if they offer Take-out, etc). Unfortunately, our `attributes` and `categories` columns aren't very user friendly and filter-able, so we might need to do a little bit of cleanup.

####Format attributes column
Taking a closer look at the `attributes` column, we see that all the attributes for a business are still stored in mulitple levels of nested JSON (which is really annoying to deal with in a DataFrame because you can't sort and do various operations with it). Luckily, the `pandas.io.json` library came equipped with a nifty little function called [`json_normalize`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html) which takes a JSON object, "flattens" any nested objects, and returns a Pandas DataFrame. So we will use this function to create a `attributes_df` DataFrame to store the attributes of a business.

In [ ]:
# Format the attributes as a list of dict objects
attributes_dict = [{'attributes': x} for x in business_df['attributes'].values]

In [ ]:
# Create a DataFrame with json_normalize
attributes_df = pd.io.json.json_normalize(attributes_dict)

Because Pandas attempted to infer the datatypes of our attributes when it loaded in the data with `json_normalize`, most of the columns came in as object types, so one thing we can do is try to convert the columns to numeric values whenever possible using the `convert_objects` function.

In [ ]:
# Convert objects to a numeric datatype if possible
attributes_df = attributes_df.convert_objects(convert_numeric=True)

Now, let's take a look at the attributes that are still non-numeric.

In [ ]:
non_numeric_attributes = attributes_df.select_dtypes(include=['object']).columns
numeric_attributes = attributes_df.select_dtypes(exclude=['object']).columns

In [ ]:
attributes_df[non_numeric_attributes].head()

The easiest way to get non-numerical/categorical data into a more consummable format is to create dummy/indicator variables for them. To do this, we can use a handy-dandy Pandas function [`get_dummies`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) to help create dummy variables. 

In [ ]:
# Create dummy variables for non-numeric attributes
dummy_vars = pd.get_dummies(attributes_df[non_numeric_attributes])

# Drop non-numeric attributes from attributes_df
attributes_df = attributes_df.drop(non_numeric_attributes, axis=1)

# Add the dummy variables to attributes_df
attributes_df = pd.merge(attributes_df, dummy_vars, left_index=True, right_index=True)

If we were to get more picky, there's probably more work we can do with cleaning the attributes, but it's in a good spot now for us to save it and merge it back with our original `business_df`

In [ ]:
# Save the list of attributes for future use
attributes = attributes_df.columns.values

In [ ]:
# Merge it with our original dataframe
business_df = pd.merge(business_df, attributes_df, left_index = True, right_index = True)

In [ ]:
# Drop our original attributes column that is no longer needed
business_df = business_df.drop('attributes', axis=1)

####Create dummy/indicator variables for categories column

Next up, if we now look at the `categories` column, we see that the categories are stored as lists. While that's easy to read, it's not actually in the most usable format if we're going to conduct any data analysis (for example, if we wanted to know how many Chinese restaurants we had in our dataset). We want to create dummy variables for the categories similar to what we did for attributes, but the categories pose an interesting dilemma because they are stored as lists. So we are going to use a slightly modified version of [`get_dummies`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) by splitting the lists up using a spring operator and then creating the dummy variables from a string.

In [ ]:
# Create dummy variables for categories
categories_df = business_df['categories'].str.join(sep=',').str.get_dummies(sep=',')

In [ ]:
# Save the list of categories for future use
categories = categories_df.columns.values

In [ ]:
# Merge it with our original dataframe
business_df = pd.merge(business_df, categories_df, left_index = True, right_index = True)

Instead of dropping the `categories` column, we're going to keep it around, but reformat it as a tuple

In [ ]:
business_df['categories'] = business_df['categories'].apply(lambda x: tuple(x))

Now we can do things like, say, filter `business_df` for all Chinese restaurants, or do a count of the number of Chinese restaurants to size up the competition.

In [ ]:
business_df[business_df['Chinese'] == 1].head()

In [ ]:
business_df['Chinese'].sum()

####So far so good!
There is definitely more clean-up work to be done with our datasets (we can continue to work with the `neighbors` or `hour` columns), but for now, we're ready to start doing some analysis!

###Descriptive Statistics
First, we might be interested in some basic descriptive statistics about our dataset. With a series of filters and statistical functions, we can do some initial exploratory analysis.

####Looking at relevant attributes
If we look at our attributes again, we see that there is a good amount of missing info (because, for example, certain attributes like `Hair Types Specialized In` simply aren't going to be applicable to any businesses other than hair salons). Since we are looking at restaurants for now, we might want to know the attributes that have the most non-null values, and therefore potentially the more important attributes for restaurants.

In [ ]:
# Count the number of non-null attributes 
nonnull_attributes_count = business_df[business_df['Restaurants'] == 1][numeric_attributes].notnull().sum()

# Sort the attribute counts
sorted_attributes = nonnull_attributes_count.order(ascending=False)

# Print the top 20
sorted_attributes[:20]

####Top restaurant categories

In [ ]:
# Count the number of restaurants are in each category
restaurant_category_counts = business_df[business_df['Restaurants'] == 1][categories].sum()

# Sort the category counts
sorted_categories = restaurant_category_counts.order(ascending=False)

# Print the top 20
sorted_categories[:20]

In [ ]:
# Get the categories that are not relevant to restaurants 
non_restaurant_categories = restaurant_category_counts[restaurant_category_counts <= 0].index.values

###Visualizing the data

We might want to also generate some plots to visualize our data. Python has a number of visualization libraries, some built on top of others. We will primarily be using [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/index.html), which is a library based on [matplotlib](http://matplotlib.org/), but feel free to check out some of the other options as well!

####Ratings Distribution

In [ ]:
sns.set(rc={"figure.figsize": (8, 4)})

data = business_df['stars']
sns.distplot(data, kde=False, bins=10)

# Add headers and labels to the plot
plt.title('Ratings Distribution')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

# Print some descriptive statistics
print "Mean: %f" % data.mean()
print "Min: %f" % data.min()
print "Max: %f" % data.max()

####Ratings Distribution for Chinese Restaurants

In [ ]:
sns.set(rc={"figure.figsize": (8, 4)})

data = business_df[business_df['Chinese'] == 1]['stars']
sns.distplot(data, kde=False, bins=10)

# Add headers and labels to the plot
plt.title('Ratings Distribution for Chinese Restaurants')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

# Print some descriptive statistics
print "Mean: %f" % data.mean()
print "Min: %f" % data.min()
print "Max: %f" % data.max()

###Analysis
There are a variety of methods that we could use to conduct the analysis we want to do, but here, we just do a very simple classifier to see what features are important for creating a good restaurant. 

####Select the data we want and format it for use with sklearn

In [ ]:
# Get just the numeric columns
numeric_only = business_df.select_dtypes(exclude=['object'])

In [ ]:
# Filter for the attributes and categories we have most information about
filtered_df = (numeric_only
                .drop('open', axis=1)
                .drop(sorted_attributes[20:].index.values, axis=1)
                .drop(['latitude', 'longitude'], axis=1)
                .drop(non_restaurant_categories, axis=1))

For now, we just replace any nan values with a 0, but in reality, there are better ways of filling in missing data

In [ ]:
# Fill any na values with 0
filtered_df = filtered_df.fillna(0)

Create training and test sets and pull out the labels (in this case we are looking at stars)

In [ ]:
# Split into data and labels
data = filtered_df[filtered_df['Restaurants'] == 1].drop('stars', axis=1)
labels = filtered_df[filtered_df['Restaurants'] == 1]['stars']

In [ ]:
# Format labels as dummy variables for classification
labels = labels.astype(str).str.get_dummies()

In [ ]:
from sklearn.cross_validation import train_test_split

# Split into test and train sets
train_data, test_data, train_labels, test_labels = train_test_split(
    data.values, labels.values, test_size=0.3, random_state=42)

####Using a random forest classifier to look at feature importance

In [ ]:
from sklearn.ensemble import RandomForestClassifier

features = data.columns.values

# Instantiate the classifier
clf = RandomForestClassifier(n_estimators = 100, max_features='auto', max_depth=4)

# Fit the classifier to our training data
clf = clf.fit(train_data,train_labels)

# look at feature importance
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
sorted_features = []
for f in range(len(indices)):
    print("%s | %f" % (features[indices[f]], importances[indices[f]]))
    sorted_features.append(features[indices[f]])

Based on this analysis, Po might try to focus on the attributes that had large importance in determining a restaurant's rating. There is, however, a lot of additional statistical and machine learning techniques we can use to better help Po conduct his analysis. 

Stay tuned for future tutorials on how Po can use techniques like natural language processing or network analysis to better help his father's restaurant!